In [1]:
import requests
from bs4 import BeautifulSoup
LINK ='https://en.wikipedia.org/wiki/Tom_Cruise'
page = requests.get(LINK)
soup = BeautifulSoup(page.content, 'html.parser')
p_tags = soup.find_all('p')
p_tags_text = [tag.get_text().strip() for tag in p_tags]

# Filter out sentences that contain newline characters '\n' or don't contain periods.
sentence_list = [sentence for sentence in p_tags_text if not '\n' in sentence]
sentence_list = [sentence for sentence in sentence_list if '.' in sentence]

content = ' '.join(sentence_list)
print(content)

Thomas Cruise Mapother IV (born July 3, 1962) is an American actor and producer. One of the world's highest-paid actors,[1] he has received various accolades, including an Honorary Palme d'Or and three Golden Globe Awards, in addition to nominations for four Academy Awards. His films have grossed over $4 billion in North America and over $11.5 billion worldwide,[2] making him one of the highest-grossing box-office stars of all time.[3] Cruise began acting in the early 1980s and made his breakthrough with leading roles in the comedy film Risky Business (1983) and action film Top Gun (1986). Critical acclaim came with his roles in the dramas The Color of Money (1986), Rain Man (1988), and Born on the Fourth of July (1989). For his portrayal of Ron Kovic in the latter, he won a Golden Globe Award and received a nomination for the Academy Award for Best Actor. As a leading Hollywood star in the 1990s, he starred in several commercially successful films, including the drama A Few Good Men (

In [2]:
from transformers import *
from bs4 import BeautifulSoup
model = pipeline("summarization", model="facebook/bart-large-cnn")


loading configuration file config.json from cache at C:\Users\anshn/.cache\huggingface\hub\models--facebook--bart-large-cnn\snapshots\3d224934c6541b2b9147e023c2f6f6fe49bd27e1\config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-large-cnn",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "force_bos_token_to_be_generated": true,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointi

In [28]:
import requests
LINK = 'https://en.wikipedia.org/wiki/Tom_Cruise'
page = requests.get(LINK)
soup = BeautifulSoup(page.content, 'html.parser')
p_tags = soup.find_all('p')
p_tags_text = [tag.get_text().strip() for tag in p_tags]

# Filter out sentences that contain newline characters '\n' or don't contain periods.
sentence_list = [sentence for sentence in p_tags_text if not '\n' in sentence]
sentence_list = [sentence for sentence in sentence_list if '.' in sentence]

content = ' '.join(sentence_list)
# result = model(content, max_length=600, min_length=500, do_sample=False)
# print(result[0]["summary_text"])

In [29]:
arr = content.split(' ')
print(len(arr)) #5000
f1k=" "
for i in range(600):
    f1k += (arr[i]+" ")
# print(len(f1k))
result = model(f1k, max_length=240, min_length=200, do_sample=False)
print(result[0]["summary_text"])
print(len(result[0]["summary_text"]))


5195
Thomas Cruise Mapother IV (born July 3, 1962) is an American actor and producer. His films have grossed over $4 billion in North America and over $11.5 billion worldwide. He began acting in the early 1980s and made his breakthrough with leading roles in the comedy film Risky Business and action film Top Gun. He has been married to actresses Mimi Rogers, Nicole Kidman, and Katie Holmes. Cruise has three children, two of whom were adopted during his marriage to Kidman and the other of whom is a biological daughter he had with Holmes. He is an outspoken advocate for the Church of Scientology, which he credits with helping him overcome dyslexia. In the 2000s, he sparked controversy with his criticisms of psychiatry and anti-depressant drugs, his efforts to promote Scientology in Europe, and a leaked video interview of him promoting Scientology. Cruise was born in Syracuse, New York, and has three sisters. One of his cousins, William Mapother, is also an actor who has appeared alongsid

In [ ]:
print(len(content))
content = content[0:4000]
result = model(content, max_length=500, min_length=400, do_sample=False)
print(result[0]["summary_text"])

In [3]:
import time
ts = time.time()
ts = str(ts)
ts

'1681375763.2889106'

In [ ]:
from flask import Flask, render_template, request
from transformers import *
import requests
from bs4 import BeautifulSoup

app = Flask('summary_app')
model = pipeline("summarization", model="facebook/bart-large-cnn")

#map url / to fuction show_index()
@app.route('/')
def show_index():
    return render_template('index.html')

#url /results is now mapped to results() function
@app.route('/resultArt', methods=['POST'])
def result1():
    if request.method == 'POST':
        lst = [x for x in request.form.values()]
        ARTICLE = lst[0]
        minln = int(request.form.get("length"))
        maxln = int(1.2*minln)
        result = model(ARTICLE, max_length=maxln, min_length=minln, do_sample=False)
        return render_template('index.html',fin_summary = result[0]["summary_text"])

@app.route('/resultLink', methods=['POST'])
def result2():
    if request.method == 'POST':
        lst = [x for x in request.form.values()]
        LINK = lst[0]
        minln = int(request.form.get("length"))
        maxln = int(1.2*minln)
        page = requests.get(LINK)
        soup = BeautifulSoup(page.content, 'html.parser')
        p_tags = soup.find_all('p')
        p_tags_text = [tag.get_text().strip() for tag in p_tags]

        # Filter out sentences that contain newline characters '\n' or don't contain periods.
        sentence_list = [sentence for sentence in p_tags_text if not '\n' in sentence]
        sentence_list = [sentence for sentence in sentence_list if '.' in sentence]

        content = ' '.join(sentence_list)

        # keeping in range.
        arr = content.split(' ')
        len(arr)
        f1k=" "
        for i in range(600):
            f1k += (arr[i]+" ")
        result = model(f1k, max_length=maxln, min_length=minln, do_sample=False)
        return render_template('index.html',fin_summary = result[0]["summary_text"])
app.run("localhost", debug=True)

200